In [1]:
import pandas as pd
import numpy as np
import os

In [2]:

'''
#Generate knockoff data using Cholesky_LU.
The code for generating ISEE Omega knockoff is implemented using R. Please make sure your computer has R installed.
'''
data_folder_path = "/ihome/alee/jis117/MB/prep_R";
X_file_name = "Ry_1.csv";
from DL.knockoff.KnockoffGenerator import KnockoffGenerator;
generator = KnockoffGenerator();

knockoff_file_path = generator.Chol_Lu_knockoff(data_folder_path, X_file_name);

print("The newly generated knockoff file is named as:")
print(knockoff_file_path);

Using TensorFlow backend.


__init__


LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [3]:
''''''
# After generating the knockoff data, run DNN
Y_file_name = 'Ry_1.csv';
X_knockoff_data = pd.read_csv(knockoff_file_path);
print(X_knockoff_data.shape)
# X_knockoff_data

#nutrient_data
original_data_Y = pd.read_csv(data_folder_path+os.path.sep+Y_file_name);
# original_data_Y

X_values = X_knockoff_data.values;
Y_values = original_data_Y.values;
    
p_num = int(X_values.shape[1] / 2);
n = X_values.shape[0];
print(X_values.shape);
print(Y_values.shape);
print(p_num);
    
X_origin = X_values[:, 0:p_num];
X_knockoff = X_values[:, p_num:];

x3D_train = np.zeros((n, p_num, 2));
x3D_train[:, :, 0] = X_origin;
x3D_train[:, :, 1] = X_knockoff;
label_train = Y_values;
    
coeff = 0.05 * np.sqrt(2.0 * np.log(p_num) / n);

n_outputs = original_data_Y.shape[1];

#Save the DNN output to the following directory.
result_dir = 'data/FMT/DNN_result/';
if not os.path.exists(result_dir):
    os.makedirs(result_dir);
    
from DL.DNN.DNN import DNN;
dnn = DNN();
model = dnn.build_DNN(p_num, n_outputs, coeff);
callback = DNN.Job_finish_Callback(result_dir,p_num);
dnn.train_DNN(model, x3D_train, label_train,callback);

(111, 274)
(111, 274)
(111, 137)
137
__init__parameters
[layer]: Input	[shape]: [None, 137, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 137, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 137, 1] 

[layer]: Flatten	[shape]: [None, None] 

[layer]: Dense	[shape]: [None, 137] 

[layer]: Dense	[shape]: [None, 137] 

[layer]: Dense	[shape]: [None, 137] 

Epoch 1/20
111/111 [==============================] - 1s 12ms/step - loss: 131.8011
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 137)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 137)
Epoch 2/20
111/111 [==============================] - 0s 158us/step - loss: 129.0148
on_epoch_end
h_local1_weight = (137, 2, 1)
h_local2_weight = (137, 1, 1)
h0 = (137, 2)
h0_abs = (137, 2)
h1 = (137, 137)
h2 = (137, 137)
h3 = (137, 137)
W1 = (137, 137)
W2 = (137, 137)
W3 = (137, 137)
Epoch 3/20
111/111 [==============================] - 0s 157us/step -

In [4]:
#Apply FDR control to DNN result
from DL.FDR.FDR_control import FDR_control;
control = FDR_control();
selected_features = control.controlFilter(data_folder_path +os.path.sep+ X_file_name, "/ihome/alee/jis117/causalDeepVASE/data/FMT/DNN_result", offset=1, q=1);
#print(selected_features)

__init__
274


In [5]:
#pd.DataFrame(selected_features).to_csv(data_folder_path+"selected_features_nyc.csv")

In [5]:
#Save the selected associations
selected_associations = [];
for ele in selected_features:
    feature = ele[0];
    w = ele[1];
    selected_associations.append({"Feature1":feature,"Feature2":"Y","W":w});
    #selected_associations.append({"Feature1":feature,"Feature2":"Y"});
pd.DataFrame(selected_associations).to_csv("data/FMT/DNN_selected_associations.csv")

In [6]:
#MGM association
XY_file_name = "nonP_XY.txt";
data_folder_path = "/ihome/alee/jis117/causalDeepVASE/data/FMT";
'''
Run MGM
Note: MGM was implemented in Java and the following Python APIs call the Java implementation.
Please restart the Python program after encountering a JVM problem.
The input data file should be ".txt" format and should also include the response variables.
Here is how the input data should look like:
X1 X2 ... Xp Y1 ... Yq
1  1  ... 1  1  ... 1
'''
from MGM.MGM import MGM
mgm = MGM();
mgm_output_file = mgm.runMGM(data_folder_path, XY_file_name,lambda_continuous_continuous = 0.3, lamda_continuous_discrete = 0.3, lamda_discrete_discrete = 0.3);
print("Please find MGM's output file as:");
mgm_output_file_path = data_folder_path+os.path.sep+mgm_output_file;
print(mgm_output_file_path);


/ihome/alee/jis117/causalDeepVASE/MGM/tetradLite.jar
Please find MGM's output file as:
/ihome/alee/jis117/causalDeepVASE/data/FMT/nonP_XY_MGM_associations.csv


In [8]:
#Run DG
#Load data
data_folder_path = "/ihome/alee/jis117/causalDeepVASE/data/FMT";
Y_file_name = 'nonP_y.csv';
X_file_name = 'nonP_x.csv';
X_data = pd.read_csv(data_folder_path+os.path.sep+X_file_name);
# X_data
Y_data = pd.read_csv(data_folder_path+os.path.sep+Y_file_name);
Y_data.columns = ["Y"];
#Merge X and Y
dataset = pd.concat([X_data, Y_data], axis=1, join='inner');
print(dataset.shape);

#Calculate the covariance matrix
cov_mat = dataset.cov();
corr_inv = np.linalg.inv(cov_mat)
corr_inv = pd.DataFrame(data=corr_inv, index=cov_mat.index,columns=cov_mat.columns)
# corr_inv.head(2)

#Convert the columns to their numerical representations
col_map = {};
col_map_rev = {};
col_list = dataset.columns.to_list();
for index,ele in enumerate(col_list):
    col_map[ele] = index;
    col_map_rev[index] = ele;
print(dataset.shape);

#https://stats.stackexchange.com/questions/13810/threshold-for-correlation-coefficient-to-indicate-statistical-significance-of-a
# t = dataset.shape[0]**(1/2)

#The data may need to be normalized if neccessary.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler();
scaled_values = scaler.fit_transform(dataset);
dataset.loc[:,:] = scaled_values;

#Initialize DG object
from causal.DegenerateGaussianScore import DegenerateGaussianScore
dg = DegenerateGaussianScore(dataset,discrete_threshold=0.2);

(92, 370)
(92, 370)


In [9]:
selected_associations_sum = [];
#Load both MGM-identified and DNN associations
MGM_associations = pd.read_csv(data_folder_path+os.path.sep+"nonP_XY_MGM_associations.csv");
for index,row in MGM_associations.iterrows():
    if row["Feature1"]=="Y" or row["Feature2"]=="Y":
        print("Found.");
        selected_associations_sum.append({"Feature1":row["Feature1"],"Feature2":row["Feature2"]});
DNN_associations = pd.read_csv(data_folder_path+os.path.sep+"DNN_selected_associations.csv");
for index,row in DNN_associations.iterrows():
    selected_associations_sum.append({"Feature1":row["Feature1"],"Feature2":row["Feature2"]});

In [9]:
#Save the selected score
selected_score = [];
    
#for ele in selected_associations_sum:
    #f1 = ele["Feature1"];
    #f2 = ele["Feature2"];
for ele in list(X_data):
    f1 = ele;
    f2 = "Y";
    
    inv_val = abs(corr_inv[f1][f2]);
    if inv_val<0.0:
        continue;
    
    n1_idx = col_map[f1];
    n2_idx = col_map[f2];
    
    s1 = dg.localScore(n1_idx,{n2_idx});
    s2 = dg.localScore(n2_idx,{n1_idx});
    diff = s1-s2;

    
    if s1<s2:
        print("Cause: "+f2+", Effect: "+f1);

    elif s1>s2: # what we want
        #print("Cause: "+f1+", Effect: "+f2);
        print("%s %d %d" %(f1,s1,s2))
        selected_score.append({"Feature1":f1,"s1":s1, "s2": s2});
        
    else:
        print("Same score.");
        
pd.DataFrame(selected_score).to_csv("data/nonP_score_abs.csv")

LKT__s__Bacteroides_vulgatus 0 -142
LKT__Unclassified -15 -142
LKT__s__Bacteroides_uniformis 14 -142
LKT__g__Bacteroides 4 -141
LKT__s__Faecalibacterium_prausnitzii -5 -141
LKT__s__Ruminococcus_Unclassified 13 -143
LKT__s__Bacteroides_Unclassified 32 -138
LKT__s__Bacteroides_ovatus -7 -143
LKT__s__Eubacterium_rectale -15 -141
LKT__s__Bacteroides_stercoris 1 -143
LKT__o__Clostridiales 21 -141
LKT__s__Bacteroides_thetaiotaomicron 2 -136
LKT__s__Bacteroides_cellulosilyticus -10 -141
LKT__s__Lachnospiraceae_bacterium -2 -143
LKT__s__Akkermansia_muciniphila -29 -142
LKT__s__Parabacteroides_distasonis 13 -141
LKT__s__Blautia_Unclassified 20 -142
LKT__s__Escherichia_coli -49 -143
LKT__s__Ruminococcaceae_bacterium -26 -142
LKT__s__Bacteroides_dorei 5 -141
LKT__s__Alistipes_onderdonkii -13 -142
LKT__s__Alistipes_putredinis -57 -142
LKT__s__Prevotella_copri -9 -136
LKT__s__Clostridium_Unclassified 13 -140
LKT__f__Lachnospiraceae 7 -141
LKT__s__Bacteroides_caccae 1 -140
LKT__s__Clostridiales_bact

In [10]:
#pd.DataFrame(selected_features).to_csv(data_folder_path+"selected_features_chi.csv")

In [16]:
print(diff)

-251.0422351373964
